# Table of Contents
 <p><div class="lev1"><a href="#CMake-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CMake</a></div><div class="lev1"><a href="#Standalone-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Standalone</a></div><div class="lev2"><a href="#Building-standalone-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Building standalone</a></div><div class="lev2"><a href="#Using-executable-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Using executable</a></div><div class="lev2"><a href="#Exercise:-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Exercise:</a></div><div class="lev2"><a href="#Opening:-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Opening:</a></div><div class="lev1"><a href="#Restart-and-Post-Mortem-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Restart and Post-Mortem</a></div><div class="lev2"><a href="#Post-Mortem-display-file-generation-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Post-Mortem display file generation</a></div><div class="lev2"><a href="#Restart-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Restart</a></div><div class="lev1"><a href="#Python-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Python</a></div><div class="lev2"><a href="#Simple-MAC_CZM-law-example-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Simple MAC_CZM law example</a></div><div class="lev2"><a href="#Post-processing-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Post-processing</a></div><div class="lev2"><a href="#Visualization-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Visualization</a></div>

# CMake

Reminder: CMake is the tool preparing the compilation. There a bunch a way to modify
the compilation process. Even if you can modify the build by passing some arguments
on the command line or directly modifyin the `CMakeCache.txt` file it is advised to
use `ccmake` or `cmake-gui` to browse through the possible options.

Here are remind the most interesting options:

* OPT ([opt], profiling, debug, check): define default compilation flags to use.
* WITH_OPENMP ([FALSE], TRUE): add compilation flags to activate multi-threading.
* BUILD_STANDALONE ([FALSE], TRUE): add the compilation of an executable.
* BUILD_Fortran_LIB or BUILD_C_LIB ([FALSE], TRUE): build a shared library (Fortran or C).
* MATLIB_VERSION ([default], none, v3): select the version of MatLib to use.
* SPARSE_LIBRARY ([mumps], none, umfpack): select the version of Sparse Linear Algebra library to use.
* EXT_FEM_VERSION ([none], Xper, tense_dd): select an external library to use for finite element computation.
* EXT_MBS_VERSION ([none], Robotran, FiberModel): select an external library to use for multi-body system computation.
* WITH_SICONOS_NUMERICS ([FALSE], TRUE): activate binding with `SiconosNumerics` library.

A very ugly one:
* GFORTRAN44_PATCHING ([FALSE]): if you need this one, upgrade the Fortran compiler !

A usefull one to consult:
* LMGC90_Fortran_FLAGS (string): **DESCRIBE** the Fortran compilation flags used to build LMGC90

Use of `cmake-gui`!

Some of these options, when modified, may build some dependencies or look for a library.
In this case, more information is available in section 5.3 in `docs/dev/cmake.html` file
(to open in a web browser).


# Standalone

* _Pros_: 

 * more compatibility with profiling tools and debuggers.
 * no need to install Python to run computation.
 * lighter build.
 
* _Cons_:

  * no customized post-processing (i.e. no Python)
  * no visualization files





## Building standalone

Just activate the `BUILD_STANDALONE`. Then use the executable named `lmgc90` in the `bin` directory
of your `build` directory. You can then use this executable instead running a Python command script.

For simplicity's sake, you must now inform your shell that you need this to be accessible as a
regular command. The easiest right now is to update the `$PATH` shell variable:

```
$> export PATH=$PATH:/.../lmgc90_dev/build/bin
$> export PATH=$PATH:/.../lmgc90_dev/src/addons
```

Notice that a second path is given at the same time. Teaser time: it will be usefull in next section !



## Using executable

Go to the to any example directory (exclude examples using non-linear newton-loop
and periodic example). Generate the DATBOX if needs be, then run the example:
For example in the channel directory:
```
$> cd channel
$> python gen_sample.py
$> lmgc90
```

Then the executable will ask for some input values. You can use the following:
```
space dim: 2
deformable: n
time step: 1.e-2
nbumber of steps: 10
theta: 0.5
detection frequency: 1
```
And leave all the following parameters to their default value
(by hitting carriage return keystroke).

You probably got an error. The usual `chipy.checkDirectories()` call is a function
available thanks to Python. Thus you must revert to an old features available in
the `lmgc90_dev/src/addons` directory which allows to create the output directories
needed by LMGC90:
```
$> mkdir-lmgc
```

In the future giving the input parameters wihtin the terminal can become bothersome,
so record in a text file the input data you want to give. Then instead run:
```
$> lmgc90 < input.dat
```



## Exercise:

Rebuild the executable using `WITH_OPENMP` variable and re-run your computation. Check the effect of multi-threading with the `OUTBOX/TIMER.OUT` file.

*DO NOT FORGET TO SET ENVIRONMENT VARIABLES FOR OPENMP*

## Opening:

It is possible to generate only a shared library of LMGC90.
Then writing ones' own standalone program (for example for non-linear deformable bodies
or to manage periodicity) and linking against the library allows to modify the
default standalone program given.

Try to generate your own executable dedicated to the channel example and taking into account
the periodicity. You will have to copy and then modify the file in `src/Sandbox/Standalone/stand_alone.f90`.

# Restart and Post-Mortem


## Post-Mortem display file generation

As described in the *cons* section, there are no visualization files.
Thus the writing of paraview files must be done after the computation.

The following commands blocks will show how to do that.
First change to `channel` directory:

In [ ]:
#first change to channel directory within the notebook

from pylmgc90 import chipy

chipy.overall_SetWorkingDirectory('channel')

Now define some parameters, like the range of file to read:

In [ ]:
# range of file to read
n_min=1
n_max=10
step = 1

chipy.checkDirectories()

# space dimension
dim = 2
mhyp = 1

# fake time step
dt = 1.

freq_display = 1
ref_radius   = 1.

Then normal reading of the DATBOX content:

In [ ]:
chipy.Initialize()
chipy.SetDimension(dim,mhyp)

#utilities_logMes('INIT TIME STEPPING')
#TimeEvolution_SetTimeStep(dt)

### model reading ###
chipy.utilities_logMes('READ BEHAVIOURS')
chipy.ReadBehaviours()
chipy.ReadModels()

chipy.utilities_logMes('READ BODIES')
chipy.ReadBodies()

chipy.utilities_logMes('LOAD BEHAVIOURS')
chipy.LoadBehaviours()

chipy.utilities_logMes('READ INI DOF')
chipy.ReadIniDof()

chipy.utilities_logMes('LOAD TACTORS')
chipy.LoadTactors()

chipy.utilities_logMes('READ INI Vloc Rloc')
chipy.ReadIniVlocRloc()


And then a loop reading the files in `OUTBOX` directory and
generating the `DISPLAY` files:

In [ ]:
chipy.OpenDisplayFiles()

for k in xrange(n_min,n_max+1,step):
    #
    chipy.utilities_logMes('READ INI DOF')
    chipy.ReadIniDof(k)

    chipy.utilities_logMes('READ INI Vloc Rloc')
    chipy.ReadIniVlocRloc(k)

    chipy.WriteDisplayFiles(freq=freq_display,ref_radius=ref_radius)

chipy.CloseDisplayFiles()
chipy.Finalize()

Now check the results in Paraview.

## Restart

Please notice that the functions used to read the `.OUT` files in `OUTBOX` directory
are the same than the ones used to read `.INI` files in `DATBOX` directory.

Thus to restart a simulation from somewhere in the time ones has only to given the
file number to read as an input to the reading functions.

For example, continuing the next five time step would be done like this:

In [ ]:
# defining restart values
file_number_to_read = 10
display_next_number = 11

In [ ]:
# other computation values
dim = 2
mhyp = 1

dt = 1e-2
nb_steps = 5
theta = 0.5

freq_detect = 1

tol = 1e-4
relax = 1.0
norm = 'Quad '
gs_it1 = 50
gs_it2 = 1000
solver_type='Stored_Delassus_Loops         '

freq_write   = 1
freq_display = 1
ref_radius = 1.


In [ ]:
# Classical initialization
chipy.Initialize()

chipy.checkDirectories()
#chipy.utilities_DisableLogMes()

chipy.SetDimension(dim,mhyp)
chipy.TimeEvolution_SetTimeStep(dt)
chipy.Integrator_InitTheta(theta)
chipy.ReadBehaviours()
chipy.ReadBodies()
chipy.LoadBehaviours()

chipy.ReadDrivenDof()
chipy.LoadTactors()

# There specifying which file to read:
chipy.ReadIniDof(file_number_to_read)
chipy.ReadIniVlocRloc(file_number_to_read)

# Here specifying which file to write:
chipy.OpenDisplayFiles(display_next_number)
chipy.OpenPostproFiles()

chipy.ComputeMass()

for k in xrange(0,nb_steps):
    chipy.IncrementStep()
    chipy.ComputeFext()
    chipy.ComputeBulk()
    chipy.ComputeFreeVelocity()
  
    chipy.SelectProxTactors(freq_detect)
    chipy.RecupRloc(0.)
    chipy.ExSolver(solver_type, norm, tol, relax, gs_it1, gs_it2)
    chipy.UpdateTactBehav()
    chipy.StockRloc()
  
    chipy.ComputeDof()
    chipy.UpdateStep()
  
    chipy.WriteOutDof(freq_write)
    chipy.WriteOutVlocRloc(freq_write)
    chipy.WriteDisplayFiles(freq_display,ref_radius)
    chipy.WritePostproFiles()

chipy.CloseDisplayFiles()
chipy.ClosePostproFiles()
chipy.Finalize()

# Python

Nevertheless, Python is still very conveninent to post-process the computation.
The aim of this section is to illustrate how to use Python to extract relevant
data from the LMGC90 database to generate some specific visualization.

## Simple MAC_CZM law example

Generate an input file with the following content:
```
3
y
5.e-6
20000
0.5
1
Quad 
1.e-4
1.
50
5
n
1
Cp_Cundall
40
```

Put this file in the `mac_czm` directory and
then run the computation:
```
$> cd mac_czm
$> python gen_sample.py
$> mkdir-lmgc
$> lmgc90 < input.dat > log
```

Please not the use of the `> log` which is important in this case since there is
very little effort in computation in this example (only one contact), and a lot
of time steps (thus a lot of log message in the terminal).


## Post-processing

Now, as in previous section let's generate, after the computation, some paraview files
and stock some values of the single interaction stored

First the initialization and data definition:

In [ ]:
import numpy as np

from pylmgc90 import chipy
chipy.overall_SetWorkingDirectory('mac_czm')

# range of file to read
n_min=1
n_max=20000
step = 1

chipy.checkDirectories()

# space dimension
dim = 3
mhyp = 0

# fake time step
dt = 5.e-6

# too many time step do display
freq_display = 100
ref_radius   = 1.e-3

Now initializing LMGC90's database:

In [ ]:
chipy.Initialize()
chipy.SetDimension(dim,mhyp)

#utilities_logMes('INIT TIME STEPPING')
#TimeEvolution_SetTimeStep(dt)

### model reading ###
chipy.utilities_logMes('READ BEHAVIOURS')
chipy.ReadBehaviours()
chipy.ReadModels()

chipy.utilities_logMes('READ BODIES')
chipy.ReadBodies()

chipy.utilities_logMes('LOAD BEHAVIOURS')
chipy.LoadBehaviours()
chipy.LoadModels()

chipy.utilities_logMes('READ INI DOF')
chipy.ReadIniDof()
chipy.ReadIniGPV()

chipy.utilities_logMes('LOAD TACTORS')
chipy.LoadTactors()

chipy.utilities_logMes('READ INI Vloc Rloc')
chipy.ReadIniVlocRloc()


Then getting, for the interaction law, its parameters.
Looping over the files to extract, at each time step,
for the interaction:
* the gap
* the adhesive stress
* the damage
* the contact length

In [ ]:

chipy.OpenDisplayFiles()

for k in xrange(n_min,n_max+1,step):
    #
    #chipy.utilities_logMes('READ INI DOF')
    chipy.ReadIniDof(k)
    chipy.ReadIniGPV(k)

    #chipy.utilities_logMes('READ INI Vloc Rloc')
    chipy.ReadIniVlocRloc(k)

    if k == 1:
        # getting, only once, the parameters of the first contact law
        law  = chipy.tact_behav_GetTactBehav(1)
        law2inter = [ law[0], 1, law[2], np.zeros([4,n_max/step]) ]

    # getting interactions state
    all_inter    = chipy.inter_handler_3D_getAll( chipy.CSASp_ID )
    all_internal = chipy.inter_handler_3D_getAllInternal( chipy.CSASp_ID )

    # extract in a numpy array some meaningful data
    law2inter[3][0, k-1] = all_inter[0,12]    # gap
    law2inter[3][1, k-1] =-all_inter[0,14]    # -rln/dt
    law2inter[3][2, k-1] = all_internal[0,4 ] # beta
    law2inter[3][3, k-1] = all_internal[0,0 ] # length

    # write display files
    chipy.WriteDisplayFiles(freq=freq_display,ref_radius=ref_radius)

chipy.CloseDisplayFiles()
chipy.Finalize()


If you write this in a script, you can save your data in a very simple
way thanks to the `pickle` module :

In [ ]:
import os, pickle
f = open( os.path.join("mac_czm","RnGapBeta.p"), "wb" )
pickle.dump(law2inter,f)
f.close()

## Visualization

The aim is now to check the behaviour of the contact law implemented within
LMGC90 and compare it with the theoric behaviour re-implemented in python.

First let's write the function describing the contact law within Python:

In [ ]:
import math

def MAC_CZM(beta, u, params, surf):
    """Compute beta (damage) and adhesive stress
       from an initial beta value and a gap
    """

    cn   = params[2]
    ct   = params[3]
    w    = params[5]
  
    nucut = u[0]*u[0] + u[2]*u[2]
    nucun = ( ( abs(u[1])+u[1] ) * 0.5 ) **2
  
    if (w - beta*(ct*nucut + cn*nucun)) < 0.:
        new_beta = w/(ct*nucut + cn*nucun)
    else:
        new_beta = beta
  
    radh = u*surf*new_beta*new_beta
    radh *= np.array([ct,cn,ct])
  
    return new_beta,radh


def MAC_CZM_maxdep(params):
    """Compute an value of gap to completely ruin the interface
    """
    return 10. * math.sqrt( params[5] / params[2] )



Let's factorize a function to plot the *reference* values computed before
and the *LMGC90* value for a data, which name is an input:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ptick
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

def do_plot(row,col,nb,title,xlabel,ylabel,x,y,xref=None,yref=None):

    plt.subplot(row,col,nb)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
  
    plt.plot(x,y,'x',label='LMGC90')
    if isinstance(xref,np.ndarray) and isinstance(yref,np.ndarray) :
        plt.plot(xref,yref,'red',label='ref')
  
    plt.legend()
    plt.gca().get_yaxis().get_major_formatter().set_powerlimits((0, 0))
    plt.gca().get_xaxis().get_major_formatter().set_powerlimits((0, 0))

If you are in a scrip you can load the previously written file:

In [ ]:
f = open( os.path.join("mac_czm","RnGapBeta.p"), "r" )
law2inter = pickle.load( f )
f.close()

Compute the *reference* value with Python using `nn` points
to describe the contact law and a range of data to display.

In [ ]:
nn = 1000

params = law2inter[2]
val    = law2inter[3]

# computed displacement at which
# the interface breaks
maxdep = MAC_CZM_maxdep(params)

b = np.zeros(nn)
r = np.zeros([nn,3])
u = np.zeros([nn,3])

u[:,1] = np.linspace(0.,maxdep,nn)

b[-1] = 1.
for i in xrange(nn):
    # specious: surf is considered constant
    b[i], r[i,:] = MAC_CZM(b[i-1],u[i,:],params,val[3,0])
    
maxn = val.shape[1]
if val[0,-1] > maxdep:
    maxn = np.argmax( val[0,:] > maxdep )


And do the plotting:

In [ ]:
do_plot(1,2,1,'Normal Adhesion','gap','Rn',val[0,:maxn],val[1,:maxn],u[:,1],r[:,1])
do_plot(1,2,2,'Damage','gap','beta',val[0,:maxn],val[2,:maxn],u[:,1],b)

plt.suptitle(law2inter[0])
plt.show()